In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
import sys

In [5]:
spark = SparkSession.builder \
    .master('local') \
    .appName('Exercise2') \
    .getOrCreate()

In [6]:
df = spark.read.csv('exampleData.csv', header=True, inferSchema=True)

In [7]:
staticSchema = df.schema

In [14]:
streamingDF = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load("/home/jovyan/*.csv")

In [15]:
streamingDF.isStreaming

True

## We cannot directly show the stream data since it is a continuous stream so we first sink it with write and then show it. 
## streamingDF.show()

In [16]:
filteredDF = streamingDF.filter((F.col('air_temperature')< 5))

In [17]:
df = filteredDF.writeStream.format('memory') \
.queryName('weatherQuery') \
.outputMode('append') \ #APPEND / UPDATE / COMPLETE - 3 output modes in spark 
.start()

In [18]:
df.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [19]:
spark.sql("""
SELECT * 
FROM weatherQuery
""")\
.show(5)

+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
|_c0|         dateTime|indicator_rain|precipitation|indicator_temp|air_temperature|indicator_wetb|wetb|dewpt|vappr|relative_humidity|   msl|indicator_wdsp|wind_speed|indicator_wddir|wind_from_direction|
+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
|298|12-jan-1990 20:00|             3|          0.0|             0|            4.9|             0| 4.3|  3.4|  7.8|               90|1025.5|             2|         1|              2|                210|
|300|12-jan-1990 22:00|             3|          0.0|             0|            3.9|             0| 3.2|  2.1|  7.1|               88|1025.9|             2|         1|              2|      

In [ ]:
# we are using memory sink output mode is append we are using the default trigger which will run the micro batch as soon as it sink we
#can se it to processingTime='2 seconds' and in this way we can config the trigger.
